In [1]:
import pandas as pd

In [2]:
import glob
import os

In [3]:
src_csv = '../metadata_by_song.csv'
data = pd.read_csv(src_csv)
print(data.shape)
data = data.set_index("songID")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../metadata_by_song.csv'

In [ ]:
def create_split(data, song_num=8, random_seed=1):
    '''
    Will return test data as an new dataframe.
    Random sample song_num songs from each Q,
    and the result of the number of clips will roughly be 1/10 in each Q.
    '''
    Q1 = data[data['DominantQ'] == 1]
    Q2 = data[data['DominantQ'] == 2]
    Q3 = data[data['DominantQ'] == 3]
    Q4 = data[data['DominantQ'] == 4]
    test_Q1 = Q1.sample(song_num, random_state=1)
    test_Q2 = Q2.sample(song_num, random_state=1)
    test_Q3 = Q3.sample(song_num, random_state=1)
    test_Q4 = Q4.sample(song_num, random_state=1)
    test_data = pd.concat([test_Q1, test_Q2, test_Q3, test_Q4])
    print(test_data.sum(axis = 0))
    return test_data

In [ ]:
test_data = create_split(data)

num_Q1       25
num_Q2       19
num_Q3       21
num_Q4       23
DominantQ    80
dtype: int64


In [ ]:
train_val = data.drop(labels = test_data.index, axis = 0, inplace = False)

In [ ]:
train_val.head()

,num_Q1,num_Q2,num_Q3,num_Q4,DominantQ
songID,,,,,
e8NQ2NH0nc8,2,0,0,0,1
HQ8ISDX6PiI,0,0,0,2,4
ZTrEoB8T9YA,0,3,0,0,2
3N2G21U7guk,3,0,0,4,4
_8v0MFBZoco,2,0,0,0,1


In [ ]:
val_data = create_split(train_val)

num_Q1       24
num_Q2       30
num_Q3       33
num_Q4       27
DominantQ    80
dtype: int64


In [ ]:
train_data = train_val.drop(labels = val_data.index, axis = 0, inplace = False)

In [ ]:
#make sure there is no song overlap between train, val, test
assert len(set(train_data.index) & set(val_data.index)) == 0
assert len(set(train_data.index) & set(test_data.index)) == 0
assert len(set(test_data.index) & set(val_data.index)) == 0

In [ ]:
train_num = train_data.sum()
val_num = val_data.sum()
test_num = test_data.sum()

In [ ]:
def count_ratio(nums, Q=1):
    total = sum([x['num_Q'+str(Q)] for x in nums])
    print('train: {}'.format(nums[0]['num_Q'+ str(Q)] / total))
    print('val  : {}'.format(nums[1]['num_Q'+ str(Q)] / total))
    print('test : {}'.format(nums[2]['num_Q'+ str(Q)] / total))


In [ ]:
nums = [train_num, val_num, test_num]
count_ratio(nums, Q=1)


train: 0.804
val  : 0.096
test : 0.1


In [ ]:
#save song level split
train_data.to_csv('../split/train_SL.csv')
val_data.to_csv('../split/val_SL.csv')
test_data.to_csv('../split/test_SL.csv')

In [ ]:
data_root = '../midis'

In [ ]:
len(data_root)

45

In [ ]:
midi_files = glob.glob(os.path.join(data_root, '*.mid'))

In [ ]:
split_dict = {}
for song in train_data.index:
    split_dict[song] = 'train'
for song in val_data.index:
    split_dict[song] = 'val'
for song in test_data.index:
    split_dict[song] = 'test'


In [ ]:
train_clips = []
val_clips = []
test_clips = []

for mid in midi_files:
    filename = mid[46:67]
    songname = filename[3:14]
    if split_dict[songname] == 'train':
        train_clips.append(filename)
    if split_dict[songname] == 'val':
        val_clips.append(filename)
    if split_dict[songname] == 'test':
        test_clips.append(filename)

In [ ]:
#save the clip lists
train_df = pd.DataFrame({'clip_name': train_clips})
train_df.to_csv('../split/train_clip.csv')

val_df = pd.DataFrame({'clip_name': val_clips})
val_df.to_csv('../split/val_clip.csv')

test_df = pd.DataFrame({'clip_name': test_clips})
test_df.to_csv('../split/test_clip.csv')